In [1]:
from pyspark import SparkFiles
from pyspark import SparkContext 
from pyspark import SQLContext
import sys
import os

In [2]:
sc = SparkContext(appName ='First Lesson', master = 'local')
sc

<SparkContext master=local appName=First Lesson>

In [3]:
# rdd = sc.textFile('file:///Users/tjmask/Desktop/test.txt')
path = 'file:///Users/tjmask/Desktop/Semester2/Spark/Outbrain_Project_PySpark/datasets/'
a = sc.textFile(path+'clicks_test.csv.zip')

In [4]:
a.take(1)

['PK\x03\x04\x14\x00\x02\x00\x08\x00�]CIZk��>�v\x08�~7\x1e\x0f\x00\x1c\x00clicks_test.csvUT\t\x00\x03�}�Wk}�Wux\x0b\x00\x01\x04�\x01\x00\x00\x04\x14\x00\x00\x00\\�I�,;Ҥ9�����\'WSH��$��\x02jT�/\x15�q\x15�@ċ/̏7fl�ڈ����\x7f��������\x7f��\x7f��\x7f���2������\\c\x0b���\u074cg]�\x1b����\\�����:��㿽�\x14�o~GXz;;�l�W����c\x15�U\\muW�\\F��y��\x1b��bm?|J�\x0f���W��ں\x15�~�b<�\x07���w��5\ue378ƻ�.n�X�s�/�7g�7W՝����%�x2v��1[�����*�2�Z�x���{���\x1be:��������}q}|�x���\x02\x1ck�"\\ߨG\x18�5\x7f(�ŭ�7+�K?�Y����kv�������������|��������4\x1e��-�s�����2�%n��\x7f���\t��{���b�q']

In [7]:
rdd.take(2)

['USE db_countries;', '-- IMPORTING DATA FROM A CSV']

In [8]:
def fuc(lines):
    lines =lines.lower()
    lines = lines.split()
    return lines

In [9]:
rdd1 = rdd.map(fuc)

In [10]:
rdd1.take(2)

[['use', 'db_countries;'], ['--', 'importing', 'data', 'from', 'a', 'csv']]

In [12]:
rdd2 = rdd.flatMap(fuc)
rdd2.take(5)

['use', 'db_countries;', '--', 'importing', 'data']

In [15]:
stopwords=['a', 'use', 'the', 'is']
rdd3 = rdd2.filter(lambda x: x not in stopwords)
rdd3.take(2)

['db_countries;', '--']

In [17]:
rdd4 = rdd3.groupBy(lambda x: x[0:3])
rdd4.take(5)

[('db_', <pyspark.resultiterable.ResultIterable at 0x114eb5fd0>),
 ('--', <pyspark.resultiterable.ResultIterable at 0x114eb5f90>),
 ('imp', <pyspark.resultiterable.ResultIterable at 0x114eb9250>),
 ('dat', <pyspark.resultiterable.ResultIterable at 0x114eb90d0>),
 ('fro', <pyspark.resultiterable.ResultIterable at 0x114eb93d0>)]

In [27]:
print([(k,list(v)) for (k, v) in rdd4.take(5)])

[('db_', ['db_countries;']), ('--', ['--', '--', '--']), ('imp', ['importing']), ('dat', ['data']), ('fro', ['from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from', 'from'])]


In [263]:
rdd3_sample = rdd3.sample(False, 0.4, 42)
print(len(rdd3.collect()), len(rdd3_sample.collect()))

742 294


In [269]:
sample1 = rdd3_sample.sample(False, 0.2, 42)
sample2 = rdd3_sample.sample(False, 0.2, 42)
sample2.take(2)

['db_countries;', 'from']

In [271]:
join_sample = sample1.join(sample2)
# join_sample.collect()

In [272]:
join_sample

PythonRDD[1171] at RDD at PythonRDD.scala:53

In [30]:
num_rdd = sc.parallelize(range(1,5))
num_rdd.reduce(lambda x,y: x*y)

24

In [41]:
sqlContext = SQLContext(sc)
sqlContext

In [46]:
## sparkconf
import pyspark
rmowers = sqlContext.read.csv('file:///Users/tjmask/Desktop/Semester2/Spark/PySpark/Datasets/RidingMowers.csv', \
                              inferSchema='true', header=True)

In [273]:
rmowers.columns

['Income', 'Lot_Size', 'Ownership']

In [277]:
rmowers.orderBy(['Income', 'Lot_Size'], ascending = True).limit(7).toPandas()[['Income', 'Lot_Size']]

,Income,Lot_Size
0,33.0,18.8
1,43.2,20.4
2,47.4,16.4
3,49.2,17.6
4,51.0,14.0
5,51.0,22.0
6,52.8,20.8


In [283]:
from pyspark.sql.functions import col
owner_groupby = rmowers.groupBy('Ownership')\
                .sum('Income')
                    

In [285]:
owner_groupby.collect()

[Row(Ownership='Owner', sum(Income)=953.6999999999999),
 Row(Ownership='Nonowner', sum(Income)=688.8)]

In [278]:
rmowers.show(5)

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
|  60.0|    18.4|    Owner|
|  85.5|    16.8|    Owner|
|  64.8|    21.6|    Owner|
|  61.5|    20.8|    Owner|
|  87.0|    23.6|    Owner|
+------+--------+---------+
only showing top 5 rows



In [48]:
rmowers.printSchema()

root
 |-- Income: double (nullable = true)
 |-- Lot_Size: double (nullable = true)
 |-- Ownership: string (nullable = true)



In [49]:
rmowers.count()

24

In [328]:
rmowers.select('Income', 'Ownership').show(5)

+------+---------+
|Income|Ownership|
+------+---------+
|  60.0|    Owner|
|  85.5|    Owner|
|  64.8|    Owner|
|  61.5|    Owner|
|  87.0|    Owner|
+------+---------+
only showing top 5 rows



In [51]:
rmowers.describe('Income').show()

+-------+------------------+
|summary|            Income|
+-------+------------------+
|  count|                24|
|   mean|           68.4375|
| stddev|19.793143575710648|
|    min|              33.0|
|    max|             110.1|
+-------+------------------+



In [52]:
rmowers.filter(rmowers.Income >= '90').show()

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
| 110.1|    19.2|    Owner|
| 108.0|    17.6|    Owner|
|  93.0|    20.8|    Owner|
+------+--------+---------+



In [329]:
rmowers.where((rmowers.Income >= 60) & (rmowers.Ownership=='Owner')).show(5)

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
|  60.0|    18.4|    Owner|
|  85.5|    16.8|    Owner|
|  64.8|    21.6|    Owner|
|  61.5|    20.8|    Owner|
|  87.0|    23.6|    Owner|
+------+--------+---------+
only showing top 5 rows



In [61]:
rmowers.registerTempTable('rmowers_df')

In [68]:
sqlContext.sql('select * from rmowers_df').show(5)

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
|  60.0|    18.4|    Owner|
|  85.5|    16.8|    Owner|
|  64.8|    21.6|    Owner|
|  61.5|    20.8|    Owner|
|  87.0|    23.6|    Owner|
+------+--------+---------+
only showing top 5 rows



In [70]:
sqlContext.sql('select min(Income) min_income from rmowers_df').show()

+----------+
|min_income|
+----------+
|      33.0|
+----------+



In [71]:
sqlContext.sql("select * from rmowers_df where Income in (select min(Income) min_income from rmowers_df)").show()

+------+--------+---------+
|Income|Lot_Size|Ownership|
+------+--------+---------+
|  33.0|    18.8| Nonowner|
+------+--------+---------+



### MLlib

In [73]:
from pyspark import SparkContext, SparkConf

In [74]:
conf = SparkConf().setMaster('local[*]').setAppName('first-ML')

In [75]:
conf

In [76]:
import pandas as pd
import numpy as np

In [88]:
toyota = pd.read_csv('/Users/tjmask/Desktop/Semester2/Spark/PySpark/Datasets/ToyotaCorolla.csv')

In [89]:
toyota.head(2)

,Id,Model,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,Fuel_Type,HP,Met_Color,...,Powered_Windows,Power_Steering,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Parking_Assistant,Tow_Bar
0,1,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13500,23,10,2002,46986,Diesel,90,1,...,1,1,0,0,0,1,0,0,0,0
1,2,TOYOTA Corolla 2.0 D4D HATCHB TERRA 2/3-Doors,13750,23,10,2002,72937,Diesel,90,1,...,0,1,0,0,0,1,0,0,0,0


In [91]:
print(toyota.shape)

(1436, 39)


In [261]:
toyota_new = toyota.drop(columns =['Id', 'Model', 'Fuel_Type', 'Color']).dropna()
# toyota_new.iloc[:, 34]

In [105]:
toyota_new.to_csv('/Users/tjmask/Desktop/Semester2/Spark/PySpark/Datasets/toyota_new.txt',\
                  sep=",", index=False, header=None)

In [106]:
points = sc.textFile('file:///Users/tjmask/Desktop/Semester2/Spark/PySpark/Datasets/toyota_new.txt')

In [119]:
points.take(1)

['13500,23,10,2002,46986,90,1,0,2000,3,4,5,210,1165,0,1,3,1,1,1,0,0,1,0,1,1,1,0,0,0,1,0,0,0,0']

In [123]:
def parse_point(line):
    values = [s for s in line.strip().split(',')]
    return LabeledPoint(values[34], values[0:33])

In [124]:
data_parsed = points.map(parse_point)
data_parsed.take(5)

[LabeledPoint(0.0, [13500.0,23.0,10.0,2002.0,46986.0,90.0,1.0,0.0,2000.0,3.0,4.0,5.0,210.0,1165.0,0.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [13750.0,23.0,10.0,2002.0,72937.0,90.0,1.0,0.0,2000.0,3.0,4.0,5.0,210.0,1165.0,0.0,1.0,3.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [13950.0,24.0,9.0,2002.0,41711.0,90.0,1.0,0.0,2000.0,3.0,4.0,5.0,210.0,1165.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [14950.0,26.0,7.0,2002.0,48000.0,90.0,0.0,0.0,2000.0,3.0,4.0,5.0,210.0,1165.0,1.0,1.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0]),
 LabeledPoint(0.0, [13750.0,30.0,3.0,2002.0,38500.0,90.0,0.0,0.0,2000.0,3.0,4.0,5.0,210.0,1170.0,1.0,1.0,3.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0])]

In [125]:
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree

In [126]:
train, test = data_parsed.randomSplit([0.7, 0.3])

In [128]:
train_model = DecisionTree.trainClassifier(train, numClasses=5, categoricalFeaturesInfo= {}, maxDepth=3)

In [129]:
predictions = train_model.predict(test.map(lambda x: x.features))

In [196]:
predictions.take(5)

[0.0, 0.0, 0.0, 0.0, 0.0]

In [147]:
LabelsAndPrediction = test.map(lambda x: x.label).zip(predictions)
LabelsAndPrediction.take(10)

[(0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (0.0, 0.0),
 (1.0, 0.0),
 (0.0, 0.0)]

In [249]:
# testErr = LabelsAndPrediction.map(lambda x: x.keys()=x.values()).count()/(predictions.count())
# # testErr

In [258]:
keys = LabelsAndPrediction.keys().collect()
values = LabelsAndPrediction.values().collect()
count=0
for i in range(410):
    if keys[i] == values[i]:
        count += 1
count

295

In [260]:
testErr = 1 - count/(predictions.count())
testErr

0.2804878048780488

In [163]:
print(train_model.toDebugString())

DecisionTreeModel classifier of depth 3 with 7 nodes
  If (feature 1 <= 31.5)
   If (feature 4 <= 33403.0)
    Predict: 0.0
   Else (feature 4 > 33403.0)
    If (feature 27 <= 0.5)
     Predict: 0.0
    Else (feature 27 > 0.5)
     Predict: 1.0
  Else (feature 1 > 31.5)
   Predict: 0.0



In [176]:
LabelsAndPrediction.collect()[0]

(0.0, 0.0)

In [210]:
data_key = sc.parallelize([('a', 1),('b', 3),('c', 2),('a', 1),('d', 2),('b', 1),('d', 3)],4)
# data_key.reduceByKey(lambda x, y: x + y).collect()
data_key.collect()

[('a', 1), ('b', 3), ('c', 2), ('a', 1), ('d', 2), ('b', 1), ('d', 3)]

In [211]:
data_key.reduceByKey(lambda x, y: x==y).collect()

[('b', False), ('c', 2), ('a', True), ('d', False)]

In [ ]:
data_key.

In [216]:
LabelsAndPrediction.filter(lambda x, y: x==y)

PythonRDD[281] at RDD at PythonRDD.scala:53

In [218]:
type(LabelsAndPrediction)

pyspark.rdd.RDD

## spark 

In [286]:
## count words
x = ['yinshihan', 'tangjie', 'yinshihan', 'yinshihan', 'tangjie']
rdd = sc.parallelize(x)
rdd0 = rdd.map(lambda word: (word,1))
rdd0.collect()

[('yinshihan', 1),
 ('tangjie', 1),
 ('yinshihan', 1),
 ('yinshihan', 1),
 ('tangjie', 1)]

In [287]:
rdd1 = rdd0.reduceByKey(lambda a,b: a+b)
rdd1.collect()

[('yinshihan', 3), ('tangjie', 2)]

## read file from local& HDFS

In [326]:

import re
rdd = sc.textFile('file:///Users/tjmask/Desktop/test.txt')
rdd0 = rdd.flatMap(lambda line: re.split(r'[, /^\][''|();>=-]', line))
rdd1 = rdd0.map(lambda word: (word,1))
rdd1 = rdd1.reduceByKey(lambda a, b: a+b)

In [327]:
rdd1.collect()

[('USE', 1),
 ('db_countries', 1),
 ('', 661),
 ('IMPORTING', 1),
 ('DATA', 1),
 ('FROM', 79),
 ('A', 4),
 ('CSV', 1),
 ('select', 1),
 ('*', 34),
 ('countries_of_the_world', 25),
 ('SHOW', 1),
 ('TABLES', 1),
 ('DESCRIBE', 5),
 ('continents', 10),
 ('cities', 3),
 ('countries', 37),
 ('attractions', 8),
 ('BASIC', 1),
 ('SELECT', 78),
 ('STATEMENTS', 1),
 ('ORDER', 14),
 ('BY', 23),
 ('area', 2),
 ('WHERE', 48),
 ('area<647500', 2),
 ('DESC', 8),
 ('COUNTRY', 1),
 ('continents.NAME', 5),
 ('continents.SURFACE', 2),
 ('AS', 10),
 ('`Name', 3),
 ('of', 4),
 ('the', 4),
 ('Country`', 3),
 ('`Size', 2),
 ('in', 3),
 ('Squared', 3),
 ('Miles`', 1),
 ('1.8*continents.SURFACE', 2),
 ('Kilometers`', 2),
 ('ROUND', 1),
 ('1000', 1),
 ('2', 4),
 ('`', 1),
 ('This', 1),
 ('is', 1),
 ('just', 1),
 ('an', 1),
 ('example', 1),
 ('Size', 1),
 ('SURFACE', 2),
 ('`SIZE', 1),
 ('IN', 2),
 ('MILES`', 1),
 ('surface', 12),
 ('population', 40),
 ('ID_COUNTRY', 2),
 ('10', 10),
 ('NAME', 2),
 ('`name', 1),

In [333]:
rdd = sc.parallelize([1,2,3,4,5])
rdd.collect()

[1, 2, 3, 4, 5]

In [334]:
rdd.getNumPartitions()

1

In [335]:
rdd.glom().collect()

[[1, 2, 3, 4, 5]]

In [337]:
rdd.map(lambda i: range(1,i)).collect()

[range(1, 1), range(1, 2), range(1, 3), range(1, 4), range(1, 5)]

In [338]:
rdd.flatMap(lambda i: range(1,i)).collect()

[1, 1, 2, 1, 2, 3, 1, 2, 3, 4]

## reduce, fold, aggregate

In [349]:

rdd.reduce(lambda a,b: a+b)

15

In [351]:
def add(a,b):
    return a/b

rdd.reduce(add)

0.008333333333333333

In [357]:
# zero=0
# op = lambda x,u : x+y
# rdd.fold(zero,op)

In [360]:
zeroValue=0
seqOp = lambda a, b:a+b
combOp = lambda x,y:x+y
rdd.aggregate(zeroValue, seqOp, combOp)

15

In [361]:
rdd.glom().collect()

[[1, 2, 3, 4, 5]]

In [362]:
## filter and distinct
rdd1 = rdd.flatMap(lambda x: (x, x+1, x+2))
rdd1.collect()

[1, 2, 3, 2, 3, 4, 3, 4, 5, 4, 5, 6, 5, 6, 7]

In [364]:
rdd1.filter(lambda x: x%2==0).collect()

[2, 2, 4, 4, 4, 6, 6]

In [366]:
rdd1.distinct().collect()

[1, 2, 3, 4, 5, 6, 7]

In [368]:
# rdd3.map(t=>(t,null)).reduceByKey(x,y=>x).map(t=>t._1)

## intersextion, union, sortBy

In [369]:
rdd0= sc.parallelize(['c','a','b','b'])
rdd1 = sc.parallelize(['a','a','b','b','d'])
rdd2 = rdd0.intersection(rdd1)
rdd2.collect()

['b', 'a']

In [370]:
rdd3= rdd0.union(rdd1)
rdd3.collect()

['c', 'a', 'b', 'b', 'a', 'a', 'b', 'b', 'd']

In [380]:
rdd3.sortBy(lambda x:x).collect()
rdd3.sortBy(lambda x:x, ascending=False).collect()

['d', 'c', 'b', 'b', 'b', 'b', 'a', 'a', 'a']

## pairRDD

In [382]:
## groupBykey, reduceByKey, aggregateByKey
y = ['yinshihan', 'tangjie','yinshihan', 'tangjie','yinshihan', 'tangjie','yinshihan', \
     'tangjie','yinshihan', 'tangjie','yinshihan', 'yinshihan', 'tangjie','yinshihan']

rdd00 = sc.parallelize(y)
rdd00 = rdd00.map(lambda word: (word,1))
rdd00.collect()

[('yinshihan', 1),
 ('tangjie', 1),
 ('yinshihan', 1),
 ('tangjie', 1),
 ('yinshihan', 1),
 ('tangjie', 1),
 ('yinshihan', 1),
 ('tangjie', 1),
 ('yinshihan', 1),
 ('tangjie', 1),
 ('yinshihan', 1),
 ('yinshihan', 1),
 ('tangjie', 1),
 ('yinshihan', 1)]

In [384]:
rdd11 = rdd00.groupByKey()
rdd11.collect()

[('yinshihan', <pyspark.resultiterable.ResultIterable at 0x117d59190>),
 ('tangjie', <pyspark.resultiterable.ResultIterable at 0x117d59210>)]

In [385]:
rdd22= rdd00.reduceByKey(lambda a, b: a+b)
rdd22.collect()

[('yinshihan', 8), ('tangjie', 6)]

In [387]:
zeroValue = 0
seqFunc = lambda x,y:x+y
combFunc = lambda x,y:x+y
rdd3 = rdd00.aggregateByKey(zeroValue,seqFunc,combFunc)
rdd3.collect()

[('yinshihan', 8), ('tangjie', 6)]

In [390]:
rdd44 = rdd00.reduceByKeyLocally(lambda x,y:x+y)
rdd44

{'tangjie': 6, 'yinshihan': 8}

## pyspark word count

In [391]:
from pyspark import SparkContext, SparkConf

In [399]:
conf =  SparkConf().setMaster('local[*]').setAppName('ysh')
sc = SparkContext.getOrCreate(conf)
sc

<SparkContext master=local appName=First Lesson>

In [400]:
m = [1,2,3,4,5]
rdd = sc.parallelize(m)
rdd.collect()

[1, 2, 3, 4, 5]

In [418]:
# path = 'hdfs://TJmaskdeMacBook-Pro.local:8080/Users/tjmask/Desktop/test.txt'
path='file:///Users/tjmask/Desktop/test.txt'
rdd = sc.textFile(path)

In [8]:
# rdd.collect()

## read multiple files 

In [23]:
path = 'file:///Users/tjmask/Desktop/test'

In [25]:
rdd = sc.wholeTextFiles(path)
# rdd.take(1)[0][1]

## RDD transformation & action

### map, flatMap, filter, distinct, sample, sortBy

In [26]:
numberRDD =  sc.parallelize(range(1,11))
numberRDD.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [27]:
numberRDD.map(lambda x: x**2).collect()

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]

In [28]:
numberRDD.filter(lambda x: x%2==0).collect()

[2, 4, 6, 8, 10]

In [50]:
flatMapRDD = numberRDD.flatMap(lambda x:(x,x**2))
flatMapRDD.collect()

[1, 1, 2, 4, 3, 9, 4, 16, 5, 25, 6, 36, 7, 49, 8, 64, 9, 81, 10, 100]

In [52]:
# list(set(flatMapRDD))

In [35]:
numberRDD.flatMap(lambda x:(x,x**2)).distinct().collect()

[1, 2, 4, 3, 9, 16, 5, 25, 6, 36, 7, 49, 8, 64, 81, 10, 100]

In [49]:
x = [1,3,5,2,3,4]
x.sort(reverse=True)

In [47]:
sampleRDD = numberRDD.sample(True, 0.7,4)
sampleRDD.collect()

[1, 2, 3, 4, 4, 5, 6, 10, 10, 10]

In [54]:
sortRDD = flatMapRDD.sortBy(keyfunc=lambda x:x, ascending=False)
sortRDD.collect()

[100, 81, 64, 49, 36, 25, 16, 10, 9, 9, 8, 7, 6, 5, 4, 4, 3, 2, 1, 1]

In [61]:
def odd(x):
    if x%2==1:
        return 2*x 
    else:
        return x

In [65]:
numberRDD\
        .map(odd)\
        .filter(lambda x: x>6)\
        .distinct()\
        .sortBy(keyfunc=lambda x:x, ascending=False)\
        .collect()     

[18, 14, 10, 8]

### PairRDD: groupByKey, sortByKey, reduceByKey, aggregateByKey, sampleByKey

In [86]:
rdd = sc.parallelize(['Hello hello ysh yinshihan', 'Hi Hello ysh TJ', 'Hi, yinshihan'])
rdd.collect()

['Hello hello ysh yinshihan', 'Hi Hello ysh TJ', 'Hi, yinshihan']

In [87]:
import re
pairRDD = rdd.flatMap(lambda x: re.split(r'[ ,]',x))\
             .map(lambda x: x.lower())\
             .map(lambda word: (word,1))        
pairRDD.collect()

[('hello', 1),
 ('hello', 1),
 ('ysh', 1),
 ('yinshihan', 1),
 ('hi', 1),
 ('hello', 1),
 ('ysh', 1),
 ('tj', 1),
 ('hi', 1),
 ('', 1),
 ('yinshihan', 1)]

In [88]:
groupRDD = pairRDD.groupByKey()
groupRDD.collect()

[('hello', <pyspark.resultiterable.ResultIterable at 0x11671b110>),
 ('ysh', <pyspark.resultiterable.ResultIterable at 0x11671b510>),
 ('yinshihan', <pyspark.resultiterable.ResultIterable at 0x11671b190>),
 ('hi', <pyspark.resultiterable.ResultIterable at 0x11671b990>),
 ('tj', <pyspark.resultiterable.ResultIterable at 0x11671b8d0>),
 ('', <pyspark.resultiterable.ResultIterable at 0x11671bfd0>)]

In [91]:
reduceRDD = pairRDD.reduceByKey(lambda x,y:x+y)
reduceRDD.collect()

[('hello', 3), ('ysh', 2), ('yinshihan', 2), ('hi', 2), ('tj', 1), ('', 1)]

In [97]:
sortRDD = pairRDD.sortByKey(ascending = True, keyfunc=lambda x:x)
sortRDD.collect()

[('', 1),
 ('hello', 1),
 ('hello', 1),
 ('hello', 1),
 ('hi', 1),
 ('hi', 1),
 ('tj', 1),
 ('yinshihan', 1),
 ('yinshihan', 1),
 ('ysh', 1),
 ('ysh', 1)]

In [99]:
zeroValue=0
seqFunc = lambda x,y:x+y
combFunc =lambda x,y:x+y
aggregateRDD = pairRDD.aggregateByKey(zeroValue, seqFunc,combFunc)
aggregateRDD.collect()

[('hello', 3), ('ysh', 2), ('yinshihan', 2), ('hi', 2), ('tj', 1), ('', 1)]

In [120]:
sampleRDD = pairRDD.sampleByKey(withReplacement=True, fractions=dict([('hello', 0.1), ('ysh', 0.2), ('yinshihan', 0.3), ('hi', 0.3), ('tj', 0.1), ('', 0)]))
sampleRDD.collect()

[('hello', 1), ('ysh', 1), ('yinshihan', 1)]

In [113]:
{('hello', 0.1), ('ysh', 0.2), ('yinshihan', 0.3), ('hi', 0.3), ('tj', 0.1), ('', 0)}

{('', 0),
 ('hello', 0.1),
 ('hi', 0.3),
 ('tj', 0.1),
 ('yinshihan', 0.3),
 ('ysh', 0.2)}

In [117]:
dict([('spam', 1), ('egg', 2), ('bar', 3)])

{'bar': 3, 'egg': 2, 'spam': 1}

### union, intersection, subtract, cartesian(正交和）

In [3]:
rdd1 = sc.parallelize([1,2,3])
rdd2 = sc.parallelize([2,3,4])

In [4]:
unionRDD = rdd1.union(rdd2)
unionRDD.collect()

[1, 2, 3, 2, 3, 4]

In [5]:
intersectionRDD = rdd1.intersection(rdd2)
intersectionRDD.collect()

[2, 3]

In [6]:
subtractRDD = rdd1.subtract(rdd2)
subtractRDD.collect()

[1]

In [7]:
subtractRDD = rdd2.subtract(rdd1)
subtractRDD.collect()

[4]

In [9]:
cartesianRDD = rdd1.cartesian(rdd2)
cartesianRDD.collect()

[(1, 2), (1, 3), (1, 4), (2, 2), (2, 3), (2, 4), (3, 2), (3, 3), (3, 4)]

### join, leftOuterJoin, rightOuterJoin, fullOuterjoin

In [17]:
rdd1 = sc.parallelize([('bob','jack'),('bob','john'), ('ysh','tj'),('ysh','ysh')])
rdd2 = sc.parallelize([('bob',10),('ysh',520),('boss',521)])

innerRDD = rdd1.join(rdd2)
innerRDD.collect()

[('bob', ('jack', 10)),
 ('bob', ('john', 10)),
 ('ysh', ('tj', 520)),
 ('ysh', ('ysh', 520))]

In [18]:
leftRDD = rdd1.leftOuterJoin(rdd2)
leftRDD.collect()

[('bob', ('jack', 10)),
 ('bob', ('john', 10)),
 ('ysh', ('tj', 520)),
 ('ysh', ('ysh', 520))]

In [19]:
rightRDD = rdd1.rightOuterJoin(rdd2)
rightRDD.collect()

[('bob', ('jack', 10)),
 ('bob', ('john', 10)),
 ('ysh', ('tj', 520)),
 ('ysh', ('ysh', 520)),
 ('boss', (None, 521))]

In [20]:
fullRDD = rdd1.fullOuterJoin(rdd2)
fullRDD.collect()

[('bob', ('jack', 10)),
 ('bob', ('john', 10)),
 ('ysh', ('tj', 520)),
 ('ysh', ('ysh', 520)),
 ('boss', (None, 521))]

### action : collect, take, first
executor 返回driver的api

### RDD to dataframe

In [62]:
# rdd = sc.textFile('file:///Users/tjmask/Desktop/test/test.txt')\
#         .map(lambda line: line.split(','))\
#         .map(lambda arr: (arr[0],arr[1]))

In [61]:
# rdd.collect()

In [63]:
# df = rdd.toDF()

In [ ]:
## way 2
from pyspark import Row

spark = SparkSession.builder\
                    .master('local[*]')\
                    .appName('ysh_sql')\
                    .getOrCreate()

rdd1 = sc.textFile('file:///Users/tjmask/Desktop/test/test.txt')\
        .map(lambda line: line.split(','))\


people = rdd1.map(lambda arr: Row[name=arr[0], age=arr[1]])

df2 = spark.createDataFrame(people)
df2.show()

In [ ]:
## 
from pyspark.sql.types import *
person = rdd1.map(lambda arr: (arr[0], arr[1]))

schema = StructType([StructField('name', StringType()),
                    StructField('age', StringType()) ])



df3  = spark.createDataFrame(person, schema)
df4 = spark.read.schema(schema).json(path)

In [ ]:
## write to Mysql
url = 'jdbc:mysql://tjmask.com:8080/ysh1314'
table = 'person'
mode = 'overwrite'
properties = {'user': 'root', 'password':'123456'}
df3.write.jdbc(url=,mode=None, properties=None )

### output: foreach foreachPartition

In [24]:
fullRDD.foreach(lambda x:x)

### saveAsTextFile

In [ ]:
from pyspark import SparkConf, SparkContext
import os, time
from pysparkWordcount import conf
from datetime import datetime

# if 'SPARK_HOME' not in os.environ:
#     os.environ['SPARK_HOME'] = '.....'
#     os.environ['PYSPARK_PYTHON'] = '.....'
# conf =  SparkConf().setMaster('local[*]').setAppName('ysh')



sc = SparkContext.getOrCreate(conf)
path = 'Datasets/RidingMowers.csv'
rdd = sc.textFile(path)
print(rdd.take(2))

## word frequency
result = rdd.flatMap(lambda line: line.split(',')).map(lambda word: (word,1)).reduceByKey(lambda a,b: a+b)

## output
# result.saveAsTextFile('Datasets/wordCount'+datetime.now().strftime('%Y-%m-%d'))

def func(iter):
    for i in iter:
        print(i)
result.foreachPartition(lambda iter:func(iter))

In [ ]:
## tian chi 天池, kesci和鲸，CCF.

### handoop 
* hdfs : 分布式储存文件(distributed saving files)
* 分布式并行计算引擎
* Yarn 统一的通用的资源调度框架

### hive
    提供了sql的开发方式,结构化
    
### hbase
    海量的非结构化数据储存， 半结构化（xml)
    
### sqoop/dataX
    sql to handoop
    
### kibana, eharts, highchart, tableau, datav

### hue
    整合大数据所有的框架实现web操作
    

### spark
    sparkcore 核心
    sparksql 交互式查询 数据来源hive
    sparkstreaming 流式计算
    mllib/ml 分布式计算机器学习
    graphX 图形之间的关系
    

saveAsHadoopDataSet(jobConf())

rdd.saveAsHadoopFile[TableOutputFormat]

In [125]:
# rdd.saveAsHadoopFile[TableoutputFormat]
sampleRDD.saveAsHadoopFile[TextOutputFormat]

NameError: name 'TextOutputFormat' is not defined

In [ ]:
transform: dstream transform to RDD

dstream.foreachRDD(rdd=>{
    
})


dstream.foreachRDD((rdd,time)=>{
    
})

数据收集层: flume, sqoop, kafka

数据储存层: HDFS, hive, hbase, kafka

数据处理层: MR, hive, spark

数据展示层: tableau javaweb

In [ ]:
### MVN repository

### Dstream transform and foreachRDD